# This notebook will be mainly used for the Captsone Project. This is the first part of this project.

In [2]:
import pandas as pd
import numpy as np

In [3]:
print('Hello Captsone Project Course!')

Hello Captsone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

###### Importing the necessary libraries for this project.

In [4]:
from bs4 import BeautifulSoup
import urllib.request
import csv
import sys

###### Simple scraper function using BeautifulSoup

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
selector = '.wikitable'

def scrape(url, selector):
    i=0
    
    with urllib.request.urlopen(url) as f:
        response = f.read().decode('utf-8')
    soup = BeautifulSoup(response, 'html.parser')
    table = soup.select(selector)
    for row in table[0].find_all('tr'):
        cell_list = []
        for cell in row.find_all(['th', 'td']):
            cell_list.append(cell.text)
        cell_list[2] = cell_list[2][:-1]
        if len(cell_list):
            
            if (i == 0):
                df = pd.DataFrame(columns=cell_list)
                i+=1
            elif(cell_list[1] == 'Not assigned'):
                continue
            elif(cell_list[2] == 'Not assigned'):
                cell_list[2] = cell_list[1]
                df.loc[i-1] = cell_list
                i+=1
            else:
                df.loc[i-1] = cell_list
                i+=1
            
    return df


df = scrape(url, selector)



In [7]:
df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [17]:
df.shape

(210, 3)